<a href="https://colab.research.google.com/github/peterbmob/DHMVADoE/blob/main/Excercises/class_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. ladda data

In [ ]:
import pandas as pd
import numpy as np

data=pd.read_csv('data_lektion.csv', sep=';')

In [ ]:
data

In [ ]:
data.loc[:,'Infill'].min()

In [ ]:
# create dictionary for parameters
inputs_labels = {'A' : data.columns[0],
                 'B' :  data.columns[1],
                 'C' :  data.columns[2]}

#create list of data for high and low.
dat = [ ('A',data.loc[:,'Infill'].min(),data.loc[:,'Infill'].max()),
        ('B',data.loc[:,'Orientation'].min(),data.loc[:,'Orientation'].max()),
        ('C',data.loc[:,'Nozzle Diameter '].min(),data.loc[:,'Nozzle Diameter '].max())]

# create pandas dataframe in a pandas dataframe
inputs_df = pd.DataFrame(dat,columns=['index','low','high'])
inputs_df = inputs_df.set_index(['index'])
inputs_df['label'] = inputs_df.index.map( lambda z : inputs_labels[z] )

#print dataframe
inputs_df

In [ ]:
# compute averages and span
inputs_df['average'] = inputs_df.apply( lambda z : ( z['high'] + z['low'])/2 , axis=1)
inputs_df['span'] = inputs_df.apply( lambda z : ( z['high'] - z['low'])/2 , axis=1)

# encode the data
inputs_df['encoded_low'] = inputs_df.apply( lambda z : ( z['low']  - z['average'] )/( z['span'] ), axis=1)
inputs_df['encoded_high'] = inputs_df.apply( lambda z : ( z['high'] - z['average'] )/( z['span'] ), axis=1)

inputs_df = inputs_df.drop(['average','span'],axis=1)

inputs_df

,low,high,label,encoded_low,encoded_high
index,,,,,
A,5.0,100.0,Infill,-1.0,1.0
B,0.0,90.0,Orientation,-1.0,1.0
C,0.2,1.0,Nozzle Diameter,-1.0,1.0


In [ ]:
import itertools
encoded_inputs= list(itertools.product([-1,1],[-1,1],[-1,1]))
encoded_inputs

In [ ]:
results=pd.DataFrame(encoded_inputs)
results=results[results.columns[::-1]]
results.columns=['A','B','C']
results

In [ ]:
real_experiment = results

var_labels = []
for var in ['A','B','C']:
    var_label = inputs_df.loc[var]['label']
    var_labels.append(var_label)
    real_experiment[var_label] = results.apply(
        lambda z : inputs_df.loc[var]['low'] if z[var]<0 else inputs_df.loc[var]['high'] ,
        axis=1)

print("The values of each real variable in the experiment:")
real_experiment[var_labels]

2. Gör experiment

In [ ]:
y=data.loc[:,'Y1':'Y4']
y_mean=y.mean(axis=1)

results['y']=y_mean
results

In [ ]:
import statsmodels.formula.api as smf


dat=results.loc[:,'A':'C']
dat['y']=results.loc[:,'y']

mod = smf.ols(formula='y ~ A + B + C + A:B + A:C + B:C', data=dat)

res = mod.fit()
res.summary()

In [ ]:
params=pd.DataFrame(res.params)
params

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
plt.rcParams.update({'font.size': 22})
# Set figure and axis
fig, ax = plt.subplots(figsize=(22,10))

# Plot bars (i.e. frequencies)
ax.set_title("Pareto Chart")
ax.set_xlabel("Parameter")
ax.set_ylabel("Frequency");
params.plot.bar(ax=ax)
ax.axhline(2.06, color="orange", linestyle="dashed")

# Second y axis (i.e. cumulative percentage)
#ax2 = ax.twinx()
#ax2.plot(effects_df.index, effects_df["cum_percentage"], color="red", marker="D", ms=7)
#params.plot(y="cum_percentage", color="red", marker="D", ms=7, ax=ax2)
#ax2.yaxis.set_major_formatter(PercentFormatter())
#ax2.set_ylabel("Cumulative Percentage");

In [ ]:
mod = smf.ols(formula='y ~ A + B + A:B ', data=dat)

res = mod.fit()
res.summary()